<a href="https://colab.research.google.com/github/ferranfont/Introduccion_al_trading_algoritmico/blob/main/Quant_momentum.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**QUANTITATIVE MOMENTUM SYSTEM**

In [181]:
import pandas as pd
import numpy as np
import requests
import matplotlib
import math
from scipy import stats # Esta libreria es nueva y la usaremos para calcular el percentil

In [182]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [183]:
# Importamos todos los tickets del SP500
stocks = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/sp_500_stocks.csv')
stocks

,Ticker
0,A
1,AAL
2,AAP
3,AAPL
4,ABBV
...,...
500,YUM
501,ZBH
502,ZBRA
503,ZION


In [184]:
# Hacemos un API a un end point de momentum
IEX_CLOUD_API_TOKEN = 'Tpk_059b97af715d417d9f49f50b51b1c448'
symbol='BA'
api_url = f'https://sandbox.iexapis.com/stable/stock/{symbol}/stats?token={IEX_CLOUD_API_TOKEN}'
data = requests.get(api_url).json()
data

{'avg10Volume': 8240060,
 'avg30Volume': 10307927,
 'beta': 1.353388191713204,
 'companyName': 'Boeing Co.',
 'day200MovingAvg': 222.1,
 'day30ChangePercent': -0.02520975750434845,
 'day50MovingAvg': 216.29,
 'day5ChangePercent': -0.015848002904906015,
 'dividendYield': 0,
 'employees': 145737,
 'exDividendDate': '',
 'float': 0,
 'marketcap': 125712206965,
 'maxChangePercent': 2.184670185457043,
 'month1ChangePercent': -0.03827821703185795,
 'month3ChangePercent': -0.02404220816076991,
 'month6ChangePercent': -0.01523570707223736,
 'nextDividendDate': '',
 'nextEarningsDate': '',
 'peRatio': -15.217391555320685,
 'sharesOutstanding': 606387373,
 'ttmDividendRate': 0,
 'ttmEPS': -14.5,
 'week52change': -0.0399610850484378,
 'week52high': 289.06,
 'week52highSplitAdjustOnly': 292.28,
 'week52low': 188.92,
 'week52lowSplitAdjustOnly': 189.93,
 'year1ChangePercent': -0.04030795457853382,
 'year2ChangePercent': -0.3932094775349765,
 'year5ChangePercent': 0.3006877073502355,
 'ytdChangePerc

In [185]:
# Consultamos un campo en particular
data['month1ChangePercent']*100

-3.827821703185795

In [186]:
round(data['peRatio'],1)

-15.2

In [187]:
my_colums = ['Ticker','Descripción', 'Rentab_1año','Rentab_6m','Rentab_1m','PER','n_acciones_a_comprar']
final_dataframe = pd.DataFrame(columns=my_colums)
final_dataframe

,Ticker,Descripción,Rentab_1año,Rentab_6m,Rentab_1m,PER,n_acciones_a_comprar


In [188]:
final_dataframe = final_dataframe.append(
                                        pd.Series([stocks, 
                                                   data['companyName'], 
                                                   round(data['year1ChangePercent'],2),
                                                   round(data['month6ChangePercent'],2),
                                                   round(data['month1ChangePercent'],2),
                                                   round(data['peRatio'],1),
                                                   'N/A'],  
                                                  index =my_colums), 
                                        ignore_index = True)
final_dataframe

,Ticker,Descripción,Rentab_1año,Rentab_6m,Rentab_1m,PER,n_acciones_a_comprar
0,Ticker 0 A 1 AAL 2 AAP 3 ...,Boeing Co.,-0.04,-0.02,-0.04,-15.2,N/A


In [189]:
# Rellenamos el dataframe en cada una de sus filas (stocks tickers) con el valor que obtenemos de la API call
for i in stocks['Ticker'][:80]: #para no ralentizar el proceso lo haremos tan sólo para las primeros 80 símbolos [:80]
  api_url = f'https://sandbox.iexapis.com/stable/stock/{i}/stats?token=Tpk_059b97af715d417d9f49f50b51b1c448'
  data = requests.get(api_url).json()
  final_dataframe = final_dataframe.append(
                                        pd.Series([i, 
                                                   data['companyName'], 
                                                   round(data['year1ChangePercent'],2),
                                                   round(data['month6ChangePercent'],2),
                                                   round(data['month1ChangePercent'],2),
                                                   round(data['peRatio'],1),
                                                   'N/A'], 
                                                  index =my_colums), 
                                        ignore_index = True)

In [190]:
final_dataframe

,Ticker,Descripción,Rentab_1año,Rentab_6m,Rentab_1m,PER,n_acciones_a_comprar
0,Ticker 0 A 1 AAL 2 AAP 3 ...,Boeing Co.,-0.04,-0.02,-0.04,-15.2,N/A
1,A,Agilent Technologies Inc.,0.05,-0.22,-0.06,17.2,N/A
2,AAL,American Airlines Group Inc,-0.04,-0.05,0.03,-3.6,N/A
3,AAP,Advance Auto Parts Inc,0.35,0.06,-0.07,11.2,N/A
4,AAPL,Apple Inc,0.31,0.15,0.01,27.6,N/A
...,...,...,...,...,...,...,...
76,C,Citigroup Inc,0.00,-0.07,0.01,6.0,N/A
77,CAG,Conagra Brands Inc,0.09,0.09,0.01,15.7,N/A
78,CAH,"Cardinal Health, Inc.",0.08,0.08,0.04,27.3,N/A
79,CARR,Carrier Global Corp,0.18,-0.20,-0.10,11.7,N/A


In [191]:
# Vamos a retirar los que tienen bajo momentum con el método .sort_value()
final_dataframe.sort_values('Rentab_1año', ascending=False, inplace=True)
final_dataframe.reset_index(inplace=True) #reseteamos el índice de nuevo
final_dataframe[:30]


,index,Ticker,Descripción,Rentab_1año,Rentab_6m,Rentab_1m,PER,n_acciones_a_comprar
0,43,APA,APA Corporation,0.67,1.05,-0.00,20.4,N/A
1,38,ANET,Arista Networks Inc,0.64,0.38,0.00,23.3,N/A
2,55,AZO,Autozone Inc.,0.62,0.18,-0.01,17.7,N/A
3,40,ANTM,Anthem Inc,0.55,0.20,-0.00,9.0,N/A
4,19,AIG,American International Group Inc,0.51,0.18,0.05,2.7,N/A
5,54,AXP,American Express Co.,0.51,0.24,0.21,9.4,N/A
6,20,AIV,Apartment Investment & Management Co.,0.47,0.08,-0.01,-36.0,N/A
7,5,ABBV,Abbvie Inc,0.46,0.25,0.07,35.6,N/A
8,12,ADM,Archer Daniels Midland Co.,0.39,0.31,0.08,8.3,N/A
9,65,BK,Bank Of New York Mellon Corp,0.38,0.08,-0.04,13.8,N/A


In [192]:
final_dataframe['Percentil_1año']=0
final_dataframe['Percentil_6meses']=0
final_dataframe['Percentil_1mes']=0
final_dataframe

,index,Ticker,Descripción,Rentab_1año,Rentab_6m,Rentab_1m,PER,n_acciones_a_comprar,Percentil_1año,Percentil_6meses,Percentil_1mes
0,43,APA,APA Corporation,0.67,1.05,-0.00,20.4,N/A,0,0,0
1,38,ANET,Arista Networks Inc,0.64,0.38,0.00,23.3,N/A,0,0,0
2,55,AZO,Autozone Inc.,0.62,0.18,-0.01,17.7,N/A,0,0,0
3,40,ANTM,Anthem Inc,0.55,0.20,-0.00,9.0,N/A,0,0,0
4,19,AIG,American International Group Inc,0.51,0.18,0.05,2.7,N/A,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...
76,49,ATVI,Activision Blizzard Inc,-0.20,-0.03,-0.01,25.0,N/A,0,0,0
77,17,AES,AES Corp.,-0.23,-0.09,-0.07,27.0,N/A,0,0,0
78,39,ANSS,Ansys Inc.,-0.25,-0.17,-0.12,57.5,N/A,0,0,0
79,63,BIIB,Biogen Inc,-0.25,-0.40,-0.09,10.0,N/A,0,0,0


In [193]:
n_valores = 50

In [194]:
for i in range(0,n_valores):
  final_dataframe.loc[i,'Percentil_1año'] = round(stats.percentileofscore(final_dataframe['Rentab_1año'],final_dataframe.loc[i,'Rentab_1año']),1)
  final_dataframe.loc[i,'Percentil_6meses'] = round(stats.percentileofscore(final_dataframe['Rentab_6m'],final_dataframe.loc[i,'Rentab_6m']),1)
  final_dataframe.loc[i,'Percentil_1mes'] = round(stats.percentileofscore(final_dataframe['Rentab_1m'],final_dataframe.loc[i,'Rentab_1m']),1)
final_dataframe[:50]

,index,Ticker,Descripción,Rentab_1año,Rentab_6m,Rentab_1m,PER,n_acciones_a_comprar,Percentil_1año,Percentil_6meses,Percentil_1mes
0,43,APA,APA Corporation,0.67,1.05,-0.00,20.4,N/A,100.0,100.0,69.1
1,38,ANET,Arista Networks Inc,0.64,0.38,0.00,23.3,N/A,98.8,97.5,69.1
2,55,AZO,Autozone Inc.,0.62,0.18,-0.01,17.7,N/A,97.5,87.0,59.9
3,40,ANTM,Anthem Inc,0.55,0.20,-0.00,9.0,N/A,96.3,89.5,69.1
4,19,AIG,American International Group Inc,0.51,0.18,0.05,2.7,N/A,94.4,87.0,88.9
5,54,AXP,American Express Co.,0.51,0.24,0.21,9.4,N/A,94.4,91.4,100.0
6,20,AIV,Apartment Investment & Management Co.,0.47,0.08,-0.01,-36.0,N/A,92.6,69.1,59.9
7,5,ABBV,Abbvie Inc,0.46,0.25,0.07,35.6,N/A,91.4,93.2,92.0
8,12,ADM,Archer Daniels Midland Co.,0.39,0.31,0.08,8.3,N/A,90.1,96.3,94.4
9,65,BK,Bank Of New York Mellon Corp,0.38,0.08,-0.04,13.8,N/A,88.9,69.1,40.7


In [195]:
final_dataframe['Media']=0
final_dataframe


,index,Ticker,Descripción,Rentab_1año,Rentab_6m,Rentab_1m,PER,n_acciones_a_comprar,Percentil_1año,Percentil_6meses,Percentil_1mes,Media
0,43,APA,APA Corporation,0.67,1.05,-0.00,20.4,N/A,100.0,100.0,69.1,0
1,38,ANET,Arista Networks Inc,0.64,0.38,0.00,23.3,N/A,98.8,97.5,69.1,0
2,55,AZO,Autozone Inc.,0.62,0.18,-0.01,17.7,N/A,97.5,87.0,59.9,0
3,40,ANTM,Anthem Inc,0.55,0.20,-0.00,9.0,N/A,96.3,89.5,69.1,0
4,19,AIG,American International Group Inc,0.51,0.18,0.05,2.7,N/A,94.4,87.0,88.9,0
...,...,...,...,...,...,...,...,...,...,...,...,...
76,49,ATVI,Activision Blizzard Inc,-0.20,-0.03,-0.01,25.0,N/A,0.0,0.0,0.0,0
77,17,AES,AES Corp.,-0.23,-0.09,-0.07,27.0,N/A,0.0,0.0,0.0,0
78,39,ANSS,Ansys Inc.,-0.25,-0.17,-0.12,57.5,N/A,0.0,0.0,0.0,0
79,63,BIIB,Biogen Inc,-0.25,-0.40,-0.09,10.0,N/A,0.0,0.0,0.0,0


In [196]:
from statistics import mean
for i in range(0,n_valores):
  final_dataframe.loc[i,'Media'] = round(mean([final_dataframe.loc[i,'Percentil_6meses'],final_dataframe.loc[i,'Percentil_1año'],final_dataframe.loc[i,'Percentil_1mes']]),1)

final_dataframe.head(22)


,index,Ticker,Descripción,Rentab_1año,Rentab_6m,Rentab_1m,PER,n_acciones_a_comprar,Percentil_1año,Percentil_6meses,Percentil_1mes,Media
0,43,APA,APA Corporation,0.67,1.05,-0.00,20.4,N/A,100.0,100.0,69.1,89.7
1,38,ANET,Arista Networks Inc,0.64,0.38,0.00,23.3,N/A,98.8,97.5,69.1,88.5
2,55,AZO,Autozone Inc.,0.62,0.18,-0.01,17.7,N/A,97.5,87.0,59.9,81.5
3,40,ANTM,Anthem Inc,0.55,0.20,-0.00,9.0,N/A,96.3,89.5,69.1,85.0
4,19,AIG,American International Group Inc,0.51,0.18,0.05,2.7,N/A,94.4,87.0,88.9,90.1
5,54,AXP,American Express Co.,0.51,0.24,0.21,9.4,N/A,94.4,91.4,100.0,95.3
6,20,AIV,Apartment Investment & Management Co.,0.47,0.08,-0.01,-36.0,N/A,92.6,69.1,59.9,73.9
7,5,ABBV,Abbvie Inc,0.46,0.25,0.07,35.6,N/A,91.4,93.2,92.0,92.2
8,12,ADM,Archer Daniels Midland Co.,0.39,0.31,0.08,8.3,N/A,90.1,96.3,94.4,93.6
9,65,BK,Bank Of New York Mellon Corp,0.38,0.08,-0.04,13.8,N/A,88.9,69.1,40.7,66.2


In [197]:
# Script para eliminar filas axis=0 o columnas axis=1
#final_dataframe.drop(['12'], axis=1)


In [198]:
# Ordenamos el dataframe segun el campo Media
final_dataframe.sort_values('Media', ascending = False, inplace = True)
final_dataframe = final_dataframe[:50]
final_dataframe

,index,Ticker,Descripción,Rentab_1año,Rentab_6m,Rentab_1m,PER,n_acciones_a_comprar,Percentil_1año,Percentil_6meses,Percentil_1mes,Media
5,54,AXP,American Express Co.,0.51,0.24,0.21,9.4,N/A,94.4,91.4,100.0,95.3
8,12,ADM,Archer Daniels Midland Co.,0.39,0.31,0.08,8.3,N/A,90.1,96.3,94.4,93.6
7,5,ABBV,Abbvie Inc,0.46,0.25,0.07,35.6,N/A,91.4,93.2,92.0,92.2
21,67,BKR,Baker Hughes Co,0.29,0.47,0.12,-70.6,N/A,74.1,98.8,98.8,90.6
4,19,AIG,American International Group Inc,0.51,0.18,0.05,2.7,N/A,94.4,87.0,88.9,90.1
0,43,APA,APA Corporation,0.67,1.05,-0.00,20.4,N/A,100.0,100.0,69.1,89.7
14,6,ABC,Amerisource Bergen Corp.,0.37,0.20,0.07,18.6,N/A,85.2,89.5,92.0,88.9
1,38,ANET,Arista Networks Inc,0.64,0.38,0.00,23.3,N/A,98.8,97.5,69.1,88.5
3,40,ANTM,Anthem Inc,0.55,0.20,-0.00,9.0,N/A,96.3,89.5,69.1,85.0
37,66,BKNG,Booking Holdings Inc,0.14,0.27,0.10,291.0,N/A,54.3,95.1,97.5,82.3


In [199]:
# Find el proyecto, basta ahora exportarlo a un Excel
final_dataframe.to_excel('portafolio_momentum.xlsx')
print('Fichero excel exportado con éxito. Mira en la carpeta "sample_data"')

Fichero excel exportado con éxito. Mira en la carpeta "sample_data"
